In [ ]:
import pandas as pd

# 读取 card.xlsx 和 CRE.xlsx 文件
card_data = pd.read_excel('/content/drive/MyDrive/card.xlsx')
cre_data = pd.read_excel('/content/drive/MyDrive/CRE.xlsx')


In [ ]:
card_data['chargeoff%'] = card_data['chargeoffs'] / card_data['loans'] * 100
cre_data['chargeoff%'] = cre_data['chargeoffs'] / cre_data['loans'] * 100


In [ ]:
from statsmodels.tsa.stattools import adfuller

# ADF 测试
result_card = adfuller(card_data['chargeoff%'])
result_cre = adfuller(cre_data['chargeoff%'])

print(f'ADF Statistic for card_data: {result_card[0]}')
print(f'p-value for card_data: {result_card[1]}')

print(f'ADF Statistic for cre_data: {result_cre[0]}')
print(f'p-value for cre_data: {result_cre[1]}')


ADF Statistic for card_data: -2.836318810006884
p-value for card_data: 0.053270234034480314
ADF Statistic for cre_data: -1.5889755284639502
p-value for cre_data: 0.4891290849015038


In [ ]:
cre_data['diff'] = cre_data['chargeoff%'].diff().dropna()

In [ ]:
import pandas_datareader.data as web

# 下载数据
unrate = web.DataReader('UNRATE', 'fred', start='2000-01-01')
oil = web.DataReader('DCOILBRENTEU', 'fred', start='2000-01-01')
gdp = web.DataReader('GDP', 'fred', start='2000-01-01')
t10y2y = web.DataReader('T10Y2Y', 'fred', start='2000-01-01')
vix = web.DataReader('VIXCLS', 'fred', start='2000-01-01')  # 波动率 VIX

In [ ]:
# 将每个宏观经济数据的索引（日期）重置为列名
unrate.reset_index(inplace=True)
oil.reset_index(inplace=True)
gdp.reset_index(inplace=True)
t10y2y.reset_index(inplace=True)
vix.reset_index(inplace=True)


In [ ]:
# 确保日期列名一致，所有数据框的日期列都命名为 'date'
unrate.rename(columns={'DATE': 'date','UNRATE': 'unrate'}, inplace=True)
oil.rename(columns={'DATE': 'date','DCOILBRENTEU': 'oil'}, inplace=True)
gdp.rename(columns={'DATE': 'date'}, inplace=True)
t10y2y.rename(columns={'DATE': 'date'}, inplace=True)
vix.rename(columns={'DATE': 'date','VIXCLS': 'vix'}, inplace=True)

In [ ]:
unrate.head(3)

,date,unrate
0,2000-01-01,4.0
1,2000-02-01,4.1
2,2000-03-01,4.0


In [ ]:
oil.head(3)

,date,oil
0,2000-01-03,NaN
1,2000-01-04,23.95
2,2000-01-05,23.72


In [ ]:
gdp.head(3)

,date,GDP,growth
0,2000-01-01,10002.179,NaN
1,2000-04-01,10247.720,0.024549
2,2000-07-01,10318.165,0.006874


In [ ]:
# 转换所有日期列为 datetime 格式，并设置日期为索引
for df in [unrate, oil, gdp, t10y2y, vix]:
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)

In [ ]:
# 将所有数据按季度末重新采样，取季度末的最后一个数据点
unrate_q = unrate.resample('QE').last()
oil_q = oil.resample('QE').last()
gdp_q = gdp.resample('QE').last()
t10y2y_q = t10y2y.resample('QE').last()
vix_q = vix.resample('QE').last()


In [ ]:
gdp_q

,GDP,growth
date,,
2000-03-31,10002.179,NaN
2000-06-30,10247.720,0.024549
2000-09-30,10318.165,0.006874
2000-12-31,10435.744,0.011395
2001-03-31,10470.231,0.003305
...,...,...
2023-06-30,27453.815,0.010656
2023-09-30,27967.697,0.018718
2023-12-31,28296.967,0.011773


In [ ]:
# 计算 GDP 增长率
gdp_q['growth'] = (gdp_q['GDP'] - gdp_q['GDP'].shift(1)) / gdp_q['GDP'].shift(1)



In [ ]:
# 确保 card 数据也以季度为单位
card_data['date'] = pd.to_datetime(card_data['date'])
card_data.set_index('date', inplace=True)
card_data_q = card_data.resample('Q').last()  # 将卡片数据按季度重新采样



<ipython-input-53-3919f7b699d6>:4: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  card_data_q = card_data.resample('Q').last()  # 将卡片数据按季度重新采样


In [ ]:
# 合并所有经济数据和冲销数据，确保日期对齐
merged_data = pd.merge(card_data_q, gdp_q[['GDP', 'growth']], left_index=True, right_index=True, how='inner')
merged_data = pd.merge(merged_data, unrate_q[['unrate']], left_index=True, right_index=True, how='inner')
merged_data = pd.merge(merged_data, oil_q[['oil']], left_index=True, right_index=True, how='inner')
merged_data = pd.merge(merged_data, t10y2y_q[['T10Y2Y']], left_index=True, right_index=True, how='inner')
merged_data = pd.merge(merged_data, vix_q[['vix']], left_index=True, right_index=True, how='inner')

In [ ]:
merged_data

,loans,chargeoffs,chargeoff%,GDP,growth,unrate,oil,T10Y2Y,vix
date,,,,,,,,,
2001-03-31,227610689,2884219,1.267172,10470.231,0.003305,4.3,23.50,0.75,28.64
2001-06-30,238509983,3159203,1.324558,10599.000,0.012299,4.5,26.21,1.17,19.06
2001-09-30,230994535,3050252,1.320487,10598.020,-0.000092,5.0,21.87,1.74,31.93
2001-12-31,252496970,3893403,1.541960,10660.465,0.005892,5.7,19.35,2.00,23.80
2002-03-31,270137409,5172643,1.914819,10783.500,0.011541,5.7,25.34,1.70,17.40
...,...,...,...,...,...,...,...,...,...
2018-12-31,916138378,9258658,1.010618,20917.867,0.005728,3.9,50.57,0.21,25.42
2019-03-31,873154649,10050661,1.151075,21111.600,0.009262,3.8,67.93,0.14,13.71
2019-06-30,894825286,10040992,1.122118,21397.938,0.013563,3.6,67.52,0.25,15.08


In [ ]:
# 步骤 9: 定义平稳性检验及差分函数
def check_stationarity_and_diff(data, column_name):
    result = adfuller(data[column_name].dropna())
    print(f'ADF Statistic for {column_name}: {result[0]}')
    print(f'p-value for {column_name}: {result[1]}')

    if result[1] > 0.05:  # 如果 p-value > 0.05，进行一阶差分
        data[f'{column_name}_diff'] = data[column_name].diff()
        print(f'Applied first differencing to {column_name}')
    else:
        print(f'{column_name} is stationary.')
    print('-' * 50)
    return data

In [ ]:
# 步骤 10: 对各个变量进行平稳性检验并差分
columns_to_check = ['unrate', 'growth', 'oil', 'T10Y2Y', 'vix']
for column in columns_to_check:
    merged_data = check_stationarity_and_diff(merged_data, column)

ADF Statistic for unrate: -1.7069987080203208
p-value for unrate: 0.42751018743968844
Applied first differencing to unrate
--------------------------------------------------
ADF Statistic for growth: -4.897972319088651
p-value for growth: 3.519268700065346e-05
growth is stationary.
--------------------------------------------------
ADF Statistic for oil: -2.387031061858582
p-value for oil: 0.1454303019492395
Applied first differencing to oil
--------------------------------------------------
ADF Statistic for T10Y2Y: -1.3972654388283334
p-value for T10Y2Y: 0.583586397100057
Applied first differencing to T10Y2Y
--------------------------------------------------
ADF Statistic for vix: -2.9832983615344686
p-value for vix: 0.036483835482600334
vix is stationary.
--------------------------------------------------


<ipython-input-65-a263035b40bd>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'{column_name}_diff'] = data[column_name].diff()
<ipython-input-65-a263035b40bd>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'{column_name}_diff'] = data[column_name].diff()
<ipython-input-65-a263035b40bd>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyd

In [ ]:
# 步骤 11: 创建滞后变量，将本季度的冲销率与上一季度的宏观经济变量对齐
merged_data['lagged_chargeoff'] = merged_data['chargeoff%'].shift(1)

# 步骤 12: 删除含有 NaN 的行（主要是由于差分和滞后造成的）
merged_data = merged_data.dropna()

<ipython-input-67-03800319369b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data['lagged_chargeoff'] = merged_data['chargeoff%'].shift(1)


In [ ]:
merged_data

,loans,chargeoffs,chargeoff%,GDP,growth,unrate,oil,T10Y2Y,vix,unrate_diff,oil_diff,T10Y2Y_diff,vix_diff,lagged_chargeoff
date,,,,,,,,,,,,,,
2001-09-30,230994535,3050252,1.320487,10598.020,-0.000092,5.0,21.87,1.74,31.93,0.5,-4.34,0.57,12.87,1.324558
2001-12-31,252496970,3893403,1.541960,10660.465,0.005892,5.7,19.35,2.00,23.80,0.7,-2.52,0.26,-8.13,1.320487
2002-03-31,270137409,5172643,1.914819,10783.500,0.011541,5.7,25.34,1.70,17.40,0.0,5.99,-0.30,-6.40,1.541960
2002-06-30,275573837,4352632,1.579479,10887.460,0.009641,5.8,25.33,1.96,25.40,0.1,-0.01,0.26,8.00,1.914819
2002-09-30,293938840,4114315,1.399718,10984.040,0.008871,5.7,29.11,1.91,39.69,-0.1,3.78,-0.05,14.29,1.579479
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31,916138378,9258658,1.010618,20917.867,0.005728,3.9,50.57,0.21,25.42,0.2,-32.15,-0.03,13.30,1.003105
2019-03-31,873154649,10050661,1.151075,21111.600,0.009262,3.8,67.93,0.14,13.71,-0.1,17.36,-0.07,-11.71,1.010618
2019-06-30,894825286,10040992,1.122118,21397.938,0.013563,3.6,67.52,0.25,15.08,-0.2,-0.41,0.11,1.37,1.151075


In [ ]:
# 步骤 13: 构建回归模型
# 使用滞后冲销率和差分后的宏观经济变量
import statsmodels.api as sm
X = merged_data[['lagged_chargeoff', 'unrate_diff', 'growth', 'oil_diff', 'T10Y2Y_diff','vix']]
X = sm.add_constant(X)  # 添加常数项
y = merged_data['chargeoff%']

# 拟合 OLS 模型
model = sm.OLS(y, X).fit()

# 输出模型结果
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             chargeoff%   R-squared:                       0.879
Model:                            OLS   Adj. R-squared:                  0.868
Method:                 Least Squares   F-statistic:                     80.76
Date:                Thu, 03 Oct 2024   Prob (F-statistic):           1.02e-28
Time:                        22:07:26   Log-Likelihood:                 30.886
No. Observations:                  74   AIC:                            -47.77
Df Residuals:                      67   BIC:                            -31.64
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                0.1941      0.089  

In [ ]:
import itertools
import statsmodels.api as sm

# 获取所有的宏观经济变量（差分后）作为因子
macro_factors = ['unrate_diff', 'growth', 'oil_diff', 'T10Y2Y_diff','vix']

# 滞后变量
merged_data['lagged_chargeoff'] = merged_data['chargeoff%'].shift(1)

# 删除含有 NaN 的行（主要是由于差分和滞后造成的）
merged_data = merged_data.dropna()

# 存储模型结果
model_results = []

# 获取所有三因子组合
three_factor_combinations = list(itertools.combinations(macro_factors, 3))

# 运行所有 AR1 + 三因子模型
for combo in three_factor_combinations:
    factors = list(combo) + ['lagged_chargeoff']

    # 构建 X 矩阵（自变量）
    X = merged_data[factors]
    X = sm.add_constant(X)  # 添加常数项

    # 构建回归模型
    y = merged_data['chargeoff%']
    model = sm.OLS(y, X).fit()

    # 存储模型结果
    model_results.append({
        'factors': combo,
        'model': model,
        'r_squared': model.rsquared
    })

# 找到 R² 最大的模型
best_model = max(model_results, key=lambda x: x['r_squared'])

# 输出最佳模型的信息
print(f"最佳模型的因子组合: {best_model['factors']}")
print(f"R-squared: {best_model['r_squared']}")
print(best_model['model'].summary())


最佳模型的因子组合: ('unrate_diff', 'growth', 'oil_diff')
R-squared: 0.878767708370747
                            OLS Regression Results                            
Dep. Variable:             chargeoff%   R-squared:                       0.879
Model:                            OLS   Adj. R-squared:                  0.872
Method:                 Least Squares   F-statistic:                     123.2
Date:                Thu, 03 Oct 2024   Prob (F-statistic):           2.15e-30
Time:                        22:08:54   Log-Likelihood:                 30.048
No. Observations:                  73   AIC:                            -50.10
Df Residuals:                      68   BIC:                            -38.64
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------